In [1]:
import pandas as pd

# New cell at index 0

path = r"../data/book_processed_output.csv"
df = pd.read_csv(path)

# Try common column names first, then try to auto-detect a 1-5 numeric column.
common = {"rating", "review_rating", "stars", "review_stars", "score"}
col = next((c for c in df.columns if c.lower() in common), None)

if col is None:
    for c in df.columns:
        s = pd.to_numeric(df[c], errors="coerce").dropna()
        if not s.empty and s.isin([1, 2, 3, 4, 5]).all():
            col = c
            break

if col is None:
    raise ValueError(f"Couldn't find a 1-5 rating column. Available columns: {list(df.columns)}")

counts = pd.to_numeric(df[col], errors="coerce").dropna().astype(int).value_counts().reindex(range(1, 6), fill_value=0).sort_index()
print(f"Rating counts for column '{col}':\n{counts}")

Rating counts for column 'rating':
rating
1      568
2     1592
3     5220
4    13152
5    21788
Name: count, dtype: int64


In [2]:
# Remap labels to 3-class sentiment: Negative (1,2) -> 0, Mixed (3) -> 1, Positive (4,5) -> 2
print("=" * 120)
print("REMAPPING LABELS TO 3-CLASS SENTIMENT")
print("=" * 120)

def remap_labels(y):
    """
    Remap 5-class ratings (1-5) to 3-class sentiment:
    1, 2 -> 0 (Negative)
    3 -> 1 (Mixed)
    4, 5 -> 2 (Positive)
    """
    return np.array([0 if rating in [1, 2] else (1 if rating == 3 else 2) for rating in y])

# Add remapped labels to dataframe
df['sentiment'] = df['rating'].apply(lambda x: 0 if x in [1, 2] else (1 if x == 3 else 2))

# Show mapping
print("\nLabel Remapping:")
print("  1, 2 stars -> 0 (Negative)")
print("  3 stars    -> 1 (Mixed)")
print("  4, 5 stars -> 2 (Positive)")

print("\nOverall Sentiment Distribution:")
sentiment_counts = df['sentiment'].value_counts().sort_index()
print(f"  Negative (0): {sentiment_counts.get(0, 0):,} reviews")
print(f"  Mixed (1):    {sentiment_counts.get(1, 0):,} reviews")
print(f"  Positive (2): {sentiment_counts.get(2, 0):,} reviews")

print("=" * 120)

REMAPPING LABELS TO 3-CLASS SENTIMENT

Label Remapping:
  1, 2 stars -> 0 (Negative)
  3 stars    -> 1 (Mixed)
  4, 5 stars -> 2 (Positive)

Overall Sentiment Distribution:
  Negative (0): 2,160 reviews
  Mixed (1):    5,220 reviews
  Positive (2): 35,836 reviews


In [3]:
!pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\kenpo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
# Required imports for 3-class sentiment classification
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

C:\Users\kenpo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = df.dropna(subset=['lemmatized_text'])
df['lemmatized_text'] = df['lemmatized_text'].astype(str)

In [6]:
# 3-Class Sentiment Classification (All Books Combined - 80/20 Train/Test)
print("=" * 120)
print("3-CLASS SENTIMENT CLASSIFICATION - COMBINED ALL BOOKS (80/20 Train/Test Split)")
print("=" * 120)

# Prepare data with sentiment labels (Negative=0, Mixed=1, Positive=2)
X_sentiment = df['lemmatized_text'].values # using lemmatized text
y_sentiment = df['sentiment'].astype(int).values

print(f"\n1. Data Summary:")
print(f"   Total reviews: {len(X_sentiment):,}")
print(f"   Sentiment distribution:")
print(f"     Negative (1-2★): {(y_sentiment == 0).sum():,} reviews")
print(f"     Mixed (3★):      {(y_sentiment == 1).sum():,} reviews")
print(f"     Positive (4-5★): {(y_sentiment == 2).sum():,} reviews")

# Split into 80/20 train/test with stratification
print(f"\n2. Splitting data (80/20 with stratification)...")
X_train_sent, X_test_sent, y_train_sent, y_test_sent = train_test_split(
    X_sentiment, y_sentiment, test_size=0.2, random_state=42, stratify=y_sentiment
)

print(f"   Training set: {len(X_train_sent):,} reviews")
print(f"   Test set: {len(X_test_sent):,} reviews")
print(f"   Training distribution:")
print(f"     Negative: {(y_train_sent == 0).sum():,} | Mixed: {(y_train_sent == 1).sum():,} | Positive: {(y_train_sent == 2).sum():,}")
print(f"   Test distribution:")
print(f"     Negative: {(y_test_sent == 0).sum():,} | Mixed: {(y_test_sent == 1).sum():,} | Positive: {(y_test_sent == 2).sum():,}")

# Load SBERT model
print(f"\n3. Loading Sentence-BERT model...")
model_sent = SentenceTransformer('all-MiniLM-L6-v2')
print(f"   ✓ Model loaded successfully (384-dimensional embeddings)")

# Generate embeddings for training set
print(f"\n4. Generating embeddings for training set...")
print(f"   Processing {len(X_train_sent):,} reviews...")
embeddings_train_sent = model_sent.encode(X_train_sent, show_progress_bar=True, batch_size=32)
print(f"   ✓ Shape: {embeddings_train_sent.shape}")

# Generate embeddings for test set
print(f"\n5. Generating embeddings for test set...")
print(f"   Processing {len(X_test_sent):,} reviews...")
embeddings_test_sent = model_sent.encode(X_test_sent, show_progress_bar=True, batch_size=32)
print(f"   ✓ Shape: {embeddings_test_sent.shape}")

# Train Random Forest classifier
print(f"\n6. Training Random Forest classifier on SBERT embeddings...")
classifier_sent = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
classifier_sent.fit(embeddings_train_sent, y_train_sent)
print(f"   ✓ Model trained successfully")

# Make predictions
print(f"\n7. Making predictions on test set...")
y_pred_sent = classifier_sent.predict(embeddings_test_sent)
accuracy_sent = accuracy_score(y_test_sent, y_pred_sent)
print(f"   ✓ Predictions completed")

# Display results
print(f"\n" + "=" * 120)
print(f"RESULTS - 3-CLASS SENTIMENT CLASSIFICATION (COMBINED)")
print(f"=" * 120)
print(f"\nAccuracy: {accuracy_sent:.4f} ({accuracy_sent*100:.2f}%)\n")

print(f"Classification Report:")
print(classification_report(y_test_sent, y_pred_sent, labels=[0, 1, 2], 
                          target_names=['Negative', 'Mixed', 'Positive'], digits=4))

# Confusion Matrix
cm_sent = confusion_matrix(y_test_sent, y_pred_sent, labels=[0, 1, 2])
print(f"Confusion Matrix:")
print(f"                Predicted Negative  Predicted Mixed  Predicted Positive")
print(f"Actual Negative     {cm_sent[0, 0]:>6}              {cm_sent[0, 1]:>6}             {cm_sent[0, 2]:>6}")
print(f"Actual Mixed        {cm_sent[1, 0]:>6}              {cm_sent[1, 1]:>6}             {cm_sent[1, 2]:>6}")
print(f"Actual Positive     {cm_sent[2, 0]:>6}              {cm_sent[2, 1]:>6}             {cm_sent[2, 2]:>6}")

print("\n" + "=" * 120)

3-CLASS SENTIMENT CLASSIFICATION - COMBINED ALL BOOKS (80/20 Train/Test Split)

1. Data Summary:
   Total reviews: 43,216
   Sentiment distribution:
     Negative (1-2★): 2,160 reviews
     Mixed (3★):      5,220 reviews
     Positive (4-5★): 35,836 reviews

2. Splitting data (80/20 with stratification)...
   Training set: 34,572 reviews
   Test set: 8,644 reviews
   Training distribution:
     Negative: 1,728 | Mixed: 4,176 | Positive: 28,668
   Test distribution:
     Negative: 432 | Mixed: 1,044 | Positive: 7,168

3. Loading Sentence-BERT model...
   ✓ Model loaded successfully (384-dimensional embeddings)

4. Generating embeddings for training set...
   Processing 34,572 reviews...


Batches: 100%|██████████| 1081/1081 [03:40<00:00,  4.89it/s]


   ✓ Shape: (34572, 384)

5. Generating embeddings for test set...
   Processing 8,644 reviews...


Batches: 100%|██████████| 271/271 [00:54<00:00,  4.99it/s]


   ✓ Shape: (8644, 384)

6. Training Random Forest classifier on SBERT embeddings...
   ✓ Model trained successfully

7. Making predictions on test set...
   ✓ Predictions completed

RESULTS - 3-CLASS SENTIMENT CLASSIFICATION (COMBINED)

Accuracy: 0.8291 (82.91%)

Classification Report:
              precision    recall  f1-score   support

    Negative     1.0000    0.0023    0.0046       432
       Mixed     0.0000    0.0000    0.0000      1044
    Positive     0.8294    0.9997    0.9066      7168

    accuracy                         0.8291      8644
   macro avg     0.6098    0.3340    0.3037      8644
weighted avg     0.7378    0.8291    0.7520      8644

Confusion Matrix:
                Predicted Negative  Predicted Mixed  Predicted Positive
Actual Negative          1                   1                430
Actual Mixed             0                   0               1044
Actual Positive          0                   2               7166



In [7]:
# 3-Class Sentiment Classification by Individual Book (80/20 Train/Test)
print("=" * 120)
print("3-CLASS SENTIMENT CLASSIFICATION - BY INDIVIDUAL BOOK (80/20 Train/Test)")
print("=" * 120)

book_results_3class = {}

for idx, (book_id, book_df) in enumerate(df.groupby('book_id'), 1):
    print(f"\n[Book {idx}/8] Processing book_id: {book_id}")
    print(f"   Total reviews: {len(book_df):,}")
    
    # Get reviews and sentiment labels
    X_book = book_df['lemmatized_text'].values
    y_book = book_df['sentiment'].astype(int).values
    
    # Show sentiment distribution
    neg_count = (y_book == 0).sum()
    mix_count = (y_book == 1).sum()
    pos_count = (y_book == 2).sum()
    print(f"   Sentiment: Negative={neg_count:,} | Mixed={mix_count:,} | Positive={pos_count:,}")
    
    # Split data (80/20 with stratification)
    try:
        X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(
            X_book, y_book, test_size=0.2, random_state=42, stratify=y_book
        )
        print(f"   Train: {len(X_train_b):,} | Test: {len(X_test_b):,}")
    except Exception as e:
        print(f"   ⚠️ Skipping (split error): {e}")
        continue
    
    # Generate embeddings
    try:
        emb_train_b = model_sent.encode(X_train_b, show_progress_bar=False, batch_size=32)
        emb_test_b = model_sent.encode(X_test_b, show_progress_bar=False, batch_size=32)
    except Exception as e:
        print(f"   ⚠️ Skipping (embedding error): {e}")
        continue
    
    # Train classifier
    try:
        clf_b = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
        clf_b.fit(emb_train_b, y_train_b)
        y_pred_b = clf_b.predict(emb_test_b)
        accuracy_b = accuracy_score(y_test_b, y_pred_b)
        
        print(f"   ✓ Accuracy: {accuracy_b:.4f} ({accuracy_b*100:.2f}%)")
        
        # Store results
        book_results_3class[book_id] = {
            'n_reviews': len(X_book),
            'n_train': len(X_train_b),
            'n_test': len(X_test_b),
            'accuracy': accuracy_b,
            'classifier': clf_b,
            'y_test': y_test_b,
            'y_pred': y_pred_b,
            'embeddings_train': emb_train_b,
            'embeddings_test': emb_test_b
        }
    except Exception as e:
        print(f"   ⚠️ Skipping (training error): {e}")
        continue

print("\n" + "=" * 120)
print("SUMMARY - 3-CLASS SENTIMENT BY INDIVIDUAL BOOK")
print("=" * 120)

if book_results_3class:
    summary_book_df = pd.DataFrame([
        {
            'Book ID': book_id,
            'Total Reviews': result['n_reviews'],
            'Train Set': result['n_train'],
            'Test Set': result['n_test'],
            'Accuracy': f"{result['accuracy']:.4f} ({result['accuracy']*100:.2f}%)"
        }
        for book_id, result in book_results_3class.items()
    ])
    print(summary_book_df.to_string(index=False))
    
    avg_accuracy_3c = np.mean([r['accuracy'] for r in book_results_3class.values()])
    print(f"\nAverage Accuracy: {avg_accuracy_3c:.4f} ({avg_accuracy_3c*100:.2f}%)")
else:
    print("No results available.")

print("=" * 120)

3-CLASS SENTIMENT CLASSIFICATION - BY INDIVIDUAL BOOK (80/20 Train/Test)

[Book 1/8] Processing book_id: 3636
   Total reviews: 5,065
   Sentiment: Negative=328 | Mixed=795 | Positive=3,942
   Train: 4,052 | Test: 1,013
   ✓ Accuracy: 0.7769 (77.69%)

[Book 2/8] Processing book_id: 136251
   Total reviews: 4,930
   Sentiment: Negative=95 | Mixed=217 | Positive=4,618
   Train: 3,944 | Test: 986
   ✓ Accuracy: 0.9371 (93.71%)

[Book 3/8] Processing book_id: 2767052
   Total reviews: 14,903
   Sentiment: Negative=460 | Mixed=1,277 | Positive=13,166
   Train: 11,922 | Test: 2,981
   ✓ Accuracy: 0.8833 (88.33%)

[Book 4/8] Processing book_id: 6218281
   Total reviews: 1,704
   Sentiment: Negative=153 | Mixed=385 | Positive=1,166
   Train: 1,363 | Test: 341
   ✓ Accuracy: 0.6891 (68.91%)

[Book 5/8] Processing book_id: 9938498
   Total reviews: 1,344
   Sentiment: Negative=136 | Mixed=353 | Positive=855
   Train: 1,075 | Test: 269
   ✓ Accuracy: 0.6543 (65.43%)

[Book 6/8] Processing book_id